In [1]:
A = open("/users/sai/Desktop/dataset/sherlock.txt",encoding="UTF-8")

data = A.read()

In [2]:
#data


In [3]:
#data.readlines()


# Tokenize data and then preprocess


In [4]:
Q = []
import re
for i in data.split(" "):
    w = re.sub("[^a-zA-Z]","",i)
    w = w.upper()
    if(w!=""):
        Q.append(w)


In [5]:
Q = Q[0:1000]


# Find Unique words


In [6]:
from numpy import unique
unique_words = unique(Q)

# Create X and Y in string format


In [7]:
len(Q)


1000

In [8]:
Xstr = []
Ystr = []
for i in range(4,len(Q),1):
    w1 = Q[i-4]
    w2 = Q[i-3]
    w3 = Q[i-2]
    w4 = Q[i-1]
    wy = Q[i]
    line = w1 + " " + w2 + " " + w3 + " " + w4
    Xstr.append(line)
    Ystr.append(wy)

In [9]:
Q[0:20]


['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN',
 'DOYLETHIS',
 'EBOOK',
 'IS',
 'FOR',
 'THE',
 'USE',
 'OF',
 'ANYONE',
 'ANYWHERE',
 'AT']

In [10]:
from pandas import DataFrame
#DataFrame([Xstr,Ystr]).T


# Declare Arrays for X and Y


In [11]:
lines = len(Xstr)
n_uniques = len(unique_words)
seq = 4

In [12]:
lines


996

In [13]:
n_uniques


515

In [14]:
seq

4

In [15]:
from numpy import zeros
X_arr = zeros((lines,n_uniques,seq),dtype="bool") 
Y_arr = zeros((lines,n_uniques))

# Data Preparation for X_arr and Y_arr


In [16]:
line_enum = list(enumerate(Xstr))


In [17]:
pos_finder = {}
for i,j in list(enumerate(unique_words)):
    pos_finder[j] = i

In [18]:
pos_finder["SHERLOCK"]


395

In [19]:
for line_no,line in line_enum:
    for seq,word in enumerate(line.split(" ")):
        X_arr[line_no,pos_finder[word],seq] = 1

In [20]:
X_arr[3,510,2]


False

In [21]:
Y_arr


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
for i,j in enumerate(Ystr):
    pos = pos_finder[j]
    Y_arr[i,pos]=1

In [23]:
Y_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
#pos_finder["SHERLOCK"]


In [25]:
#Y_arr[1,11361]


In [26]:
X_arr.shape


(996, 515, 4)

# Create a NN


In [27]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.activations import softmax
from keras.layers.core import Activation

In [28]:
nn = Sequential()
nn.add(LSTM(200,input_shape=(515,4)))
nn.add(Dense(515))
nn.add(Activation("softmax"))

In [29]:
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model = nn.fit(X_arr,Y_arr,epochs=2,validation_split=0.01)

Epoch 1/2
31/31 [==============================] - 59s 2s/step - loss: 6.0902 - accuracy: 0.0477 - val_loss: 6.9258 - val_accuracy: 0.0000e+00
Epoch 2/2
31/31 [==============================] - 54s 2s/step - loss: 5.7317 - accuracy: 0.0548 - val_loss: 7.3178 - val_accuracy: 0.0000e+00


In [38]:
q = input("Enter sentence:(4 words)")
q = q.upper()

Enter sentence:(4 words)adventures of sherlock holmes


In [39]:
q

'ADVENTURES OF SHERLOCK HOLMES'

In [40]:
inp = zeros((1,515,4),dtype="bool")

for word_no,word in enumerate(q.split(" ")):
    inp[0,pos_finder[word],word_no]=1

In [41]:
import pandas as pd
B = pd.DataFrame(zip(unique_words,list(nn.predict(inp)[0])),columns=["Word","Prob"])
x = B.Prob.max()


In [42]:
B[B.Prob == x].Word.values[0]


'THE'